# Domain Risk Training and Evaluation

This notebook trains the domain risk model and evaluates PR/confusion metrics.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from sentineldns.features.domain_features import build_domain_feature_matrix
from sentineldns.models.domain_risk import train_domain_risk_model

csv_path = Path('../data/processed/labeled_domains.csv')
df = pd.read_csv(csv_path)
df['label'].value_counts()

In [ ]:
metrics = train_domain_risk_model(csv_path)
metrics

In [ ]:
# Basic false-positive / true-positive review
df_mal = df[df.label == 1].head(20)
df_ben = df[df.label == 0].head(20)
display(df_mal)
display(df_ben)